### **Подготовка данных**
***
**Загрузка и обработка текста**

In [1]:
import pandas as pd
import numpy as np

Для подготовки обучающих данных, были загружены датасеты с негативными комментариями:

Russian Language Toxic Comments Dataset (файл rltd.csv) - основной датасет для работы с токсичными комментариями на русском языке. Опубликован на Kaggle в 2019 году и содержит 14,412 комментариев, из которых 4,826 помечены как токсичные, а 9,586 — как нетоксичные

MCA Workshop - Toxic Comments Dataset - датасет токсичных комментариев, собранный во время первого воркшопа Математического центра в Академгородке. Содержит размеченные комментарии из социальной сети ВКонтакте

Объеденим датасеты и возьмем исключительно тексты комментариев, для последующей разметки

In [2]:
df1 = pd.read_csv('C:/Users/kpodd/OneDrive/Desktop/ml/NER/data/rltcd.csv')
df2 = pd.read_csv('C:/Users/kpodd/OneDrive/Desktop/ml/NER/data/mcaw.csv')
print(df1.columns, df2.columns)
df1, df2 = df1['comment'], df2['message']
df = pd.DataFrame(pd.concat([df1, df2], axis=0), columns=['text'])
print(df.info)

Index(['comment', 'toxic'], dtype='object') Index(['Unnamed: 0', 'source', 'text', 'message', 'sex', 'decent', 'moral',
       'person'],
      dtype='object')
<bound method DataFrame.info of                                                    text
0                  Верблюдов-то за что? Дебилы, бл...\n
1     Хохлы, это отдушина затюканого россиянина, мол...
2                             Собаке - собачья смерть\n
3     Страницу обнови, дебил. Это тоже не оскорблени...
4     тебя не убедил 6-страничный пдф в том, что Скр...
...                                                 ...
3799                 [id584786137|Μάρκ], в этом с(м)ысл
3800  [id510533373|Sasha], «правильные», а судьи кто...
3801  [id88928627|Владислав], правильные это греческ...
3802  [id510533373|Sasha], почему не римский стандар...
3803  [id88928627|Владислав], поддерживаю этнос варв...

[18216 rows x 1 columns]>


***

In [3]:
# Удаление пустых строк и пробелов
df = df[df["text"].str.strip() != ""]
df = df.dropna()
display(df.head(3))
print(df.isnull().sum())

,text
0,"Верблюдов-то за что? Дебилы, бл...\n"
1,"Хохлы, это отдушина затюканого россиянина, мол..."
2,Собаке - собачья смерть\n


text    0
dtype: int64


In [4]:
# Загрузим файл, содержащий перечень нецензурных слов русского языка (2300 слов)
# Файл был взят с - https://gitverse.ru/gen/russian_ban_words
with open('C:/Users/kpodd/OneDrive/Desktop/ml/NER/data/ru_curse_words.txt', 'r', encoding='utf-8') as f:
    bad_words = set([line.strip().lower() for line in f if line.strip()])

In [5]:
import re
def clean_text(text):
    text = re.sub(r'[^а-яА-ЯёЁ\s\-.,!?;:]', '', text) # Удаление нерусских слов
    text = str(text).lower() # Lowercase
    text = re.sub(r'http\S+', '', text) # HTML
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[\U00010000-\U0001FFFF]', '', text) # Emoji
    text = re.sub(r'[^\w\s]', ' ', text) # Пунктуация
    text = re.sub(r'\d+', '', text) # Числа
    text = re.sub(r'\s+', ' ', text).strip() # Extra spaces
    return text
    
df['clean_text'] = df['text'].apply(clean_text)
display(df.head(5))
print(df.info)

,text,clean_text
0,"Верблюдов-то за что? Дебилы, бл...\n",верблюдов то за что дебилы бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",хохлы это отдушина затюканого россиянина мол в...
2,Собаке - собачья смерть\n,собаке собачья смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",страницу обнови дебил это тоже не оскорбление ...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",тебя не убедил страничный пдф в том что скрипа...


<bound method DataFrame.info of                                                    text  \
0                  Верблюдов-то за что? Дебилы, бл...\n   
1     Хохлы, это отдушина затюканого россиянина, мол...   
2                             Собаке - собачья смерть\n   
3     Страницу обнови, дебил. Это тоже не оскорблени...   
4     тебя не убедил 6-страничный пдф в том, что Скр...   
...                                                 ...   
3799                 [id584786137|Μάρκ], в этом с(м)ысл   
3800  [id510533373|Sasha], «правильные», а судьи кто...   
3801  [id88928627|Владислав], правильные это греческ...   
3802  [id510533373|Sasha], почему не римский стандар...   
3803  [id88928627|Владислав], поддерживаю этнос варв...   

                                             clean_text  
0                         верблюдов то за что дебилы бл  
1     хохлы это отдушина затюканого россиянина мол в...  
2                                 собаке собачья смерть  
3     страницу обнови дебил

In [6]:
df.to_csv("data_clean.csv", index=False)

***
Токенизация и BIO-разметка

In [7]:
from transformers import AutoTokenizer
from tqdm.autonotebook import tqdm

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [8]:
# Токенизация и разметка
def tokenize_and_label(text, bad_words, truncation=True, padding = "max_length", max_length = 256):
    tokens = tokenizer.tokenize(text)
    labels = []

    current_word_tokens = []
    current_word = ''
    output_tokens = []
    output_labels = []

    for tok in tokens:
        if tok.startswith("##"): # учет сплитов с ##
            sub_tok = tok[2:]
            current_word += sub_tok
            current_word_tokens.append(tok)
        else:
            if current_word_tokens:
                word_lower = current_word.lower()
                label_seq = ['B-PRF'] + ['I-PRF'] * (len(current_word_tokens) - 1) if word_lower in bad_words else ['O'] * len(current_word_tokens)
                output_tokens.extend(current_word_tokens)
                output_labels.extend(label_seq)
            # начинаем новое слово
            current_word = tok
            current_word_tokens = [tok]
    # последнее слово
    if current_word_tokens:
        word_lower = current_word.lower()
        label_seq = ['B-PRF'] + ['I-PRF'] * (len(current_word_tokens) - 1) if word_lower in bad_words else ['O'] * len(current_word_tokens)
        output_tokens.extend(current_word_tokens)
        output_labels.extend(label_seq)

    return output_tokens, output_labels

In [9]:
token_sequences = []
label_sequences = []
df = pd.read_csv('C:/Users/kpodd/OneDrive/Desktop/ml/NER/data/data_clean.csv')
texts = df['clean_text'].dropna().tolist()

In [10]:
for text in tqdm(texts):
    tokens, labels = tokenize_and_label(text, bad_words)
    token_sequences.append(tokens)
    label_sequences.append(labels)

assert all(len(t) == len(l) for t, l in zip(token_sequences, label_sequences)) # ошибка в длине токенов и меток

  0%|          | 0/18134 [00:00<?, ?it/s]

In [11]:
label_df = pd.DataFrame({
    'tokens': token_sequences,
    'labels': label_sequences
})
label_df.to_csv('C:/Users/kpodd/OneDrive/Desktop/ml/NER/data/data_label.csv', index=False)

In [12]:
# Уже на первом примере можно увидеть, как работает разметка
display(label_df.head(1))

,tokens,labels
0,"[верблю, ##дов, то, за, что, дебил, ##ы, бл]","[O, O, O, O, O, B-PRF, I-PRF, B-PRF]"


In [13]:
# Статистика разметки
from collections import Counter
from itertools import chain
print(Counter(chain.from_iterable(label_df['labels'])))

Counter({'O': 539780, 'B-PRF': 3647, 'I-PRF': 3350})


In [14]:
label_df.to_pickle("C:/Users/kpodd/OneDrive/Desktop/ml/NER/data/data_label.pkl")